# Acesso ao SD

> Nota: esse Notebook usa o [kernel Bash](https://pypi.org/project/bash_kernel/)

Objetivo: rodar instâncias do servidor JupyterLab (JL) no supercomputador Santos Dumont (SD), e desta forma ter acesso aos recursos do SD através do cliente JL (web-based IDE) rodando no navegador web na máquina local.

## Exemplos de configuração do ambiente

### Na máquina local

Pré-requisito: distribuição Python (Anaconda ou Intel) instalada e funcionando na máquina local

* https://www.anaconda.com/products/individual
* https://software.intel.com/content/www/us/en/develop/tools/oneapi/components/distribution-for-python.html


### Exemplo de instalação do Jupyter-Lab

Distribuição Intel (que inclui outros compiladores e ferramentas Intel)

* https://software.intel.com/content/www/us/en/develop/documentation/installation-guide-for-intel-oneapi-toolkits-linux/top/installation/install-using-package-managers/apt.html

```
$ sudo apt install intel-basekit intel-hpckit
$ source /opt/intel/oneapi/setvars.sh
$ sudo chmod -R 777 /opt/intel/oneapi/intelpython/latest/envs
$ conda install jupyterlab
$ jupyter-lab
```


### Exemplos de configuração do VPN, SSH, e JupyterLab

* https://github.com/efurlanm/hpc/blob/master/sd/access_configuring.ipynb
* https://www.hostinger.com.br/tutoriais/conexao-ssh-sem-senha
* https://github.com/efurlanm/hpc/blob/master/sd/access_using_jupyterlab.ipynb

O SSH foi configurado para chave ao invés de senha, tanto na máquina local quanto no SD para acessar o sdumont18 (\~/.ssh/id_sd_ed25519). Além disso os arquivos `~/.ssh/config` foram configurados:

* Na máquina local:
        Host *
            ServerAliveInterval 120
        Host sd
            HostName login.sdumont.lncc.br
            User <NOME_DO_USUARIO>
            ControlMaster auto
            ControlPath ~/.ssh/remote_sd
* No SD:
        Host *
            ServerAliveInterval 120
        Host sdumont18
            Identityfile ~/.ssh/id_sd_ed25519

### Configuração deste Notebook

Esta célula deve ser executada sempre. Como alternativa o `.basrc` pode ser configurado:

In [1]:
bind 'set enable-bracketed-paste off'

## Acessando o Santos Dumont

* Primeiro passo: usando o Gerenciador de Rede, fazer a conexão ao VPN Cisco usando vpnc

Ao fazer login, uma máquina de login é selecionada aleatoriamente (geralmente sdumont11, sdumont13, ou sdumont14). O comando a seguir abre uma conexão mestre ("SSH Socket") em um dos nós, de tal forma que todos os comandos subsequentes sejam realizados neste mesmo nó:

In [2]:
ssh -Nf sd    # não esquecer de ligar o VPN antes

Mostra em qual nó estamos conectados:

In [3]:
ssh sd 'hostname'

sdumont14


## Primeira sessão

* no nó de login B710
* o comando `ssh` a seguir, executa os seguintes comandos no SD:
    * carrega o módulo anaconda
    * carrega uma instância do servidor JupyterLab, que fica ativo na porta 20210 (pode escolher outra)
* uma instância JupyterLab fica rodando na máquina de login onde a conexão mestre está apontando

In [4]:
ssh -T sd <<'EOF'
module load anaconda3
jupyter-lab --no-browser --port=20210 --ip=0.0.0.0 > ~/20210.log 2>&1 &
EOF

## Segunda sessão

* acesso ao nó Sequana (sdumont18)
* entra no nó Sequana sdumont18 usando o comando ssh que é executado na máquina de login
* carrega uma nova instância do servidor JupyterLab, que fica ativo no nó sdumont18 na porta 20211

In [14]:
ssh -T sd <<'EOF1'
ssh -T sdumont18 <<'EOF2'
module load anaconda3
jupyter-lab --no-browser --port=20211 --ip=0.0.0.0 > ~/20211.log 2>&1 &
EOF2
EOF1

## Encaminhamento de portas

Localmente o acesso é feito usando as portas 8889 e 8890 (pode escolher outras)

B710:

In [5]:
ssh -NfTL 8889:localhost:20210 sd

Sequana:

In [15]:
ssh -NfTL 8890:sdumont18:20211 sd

## Montagem do sistema de arquivos remoto

* é independente do Jupyter
* é importante criar os diretórios `/pr` e `/scratch` com `chmod -R 777` no diretório raiz da máquina local, para evitar links quebrados

In [6]:
sshfs sd:/prj     /prj     -o workaround=rename,uid=1000,gid=1000
sshfs sd:/scratch /scratch -o workaround=rename,uid=1000,gid=1000

## Links para os clientes JupyterLab na máquina local

* se não aparecer nada ou aparecer em branco, tem que esperar um pouco para o servidor subir (pode demorar minutos)
* se assim mesmo o link não aparecer, ou der erro: mata os processos e tenta de novo

B710:

In [13]:
tmp=$(ssh sd 'cat ~/20210.log | grep "     or"')
echo ${tmp/#*'20210'/'http://localhost:8889'}

http://localhost:8889/?token=a4f65f2933f074b73b4bb98b57013a6169dc4421888a9957


Sequana:

In [16]:
tmp=$(ssh sd 'cat ~/20211.log | grep "     or"')
echo ${tmp/#*'20211'/'http://localhost:8890'}

http://localhost:8890/?token=e00710fd69c5a3c43548071ff70fe8d7b23f4115c13e56e0


# Sair

* como o `/scratch` é temporário, fazer a cópia dos arquivos para `/prj` ou para a máquina local
* fazer backup dos arquivos na máquina local

Desmonta o sistema de arquivos remoto, e encerra o encaminhamento de portas:

In [21]:
sudo umount -fl /prj /scratch

In [ ]:
pkill -f 8889:

In [ ]:
pkill -f 8890:

### Encerra a conexão

Uma forma é usar o menu do cliente JupyterLab e selecionar "File > Shut Down" para cada instância do servidor, e em seguida encerrar a conexão mestre, e depois desligar o VPN:

In [24]:
ssh -O exit sd

Exit request sent.


Outra forma é matar os processos e em seguida encerrar a conexão mestre:

In [20]:
ssh sd 'pkill -u ${USER} -f jupyter'

In [21]:
ssh sd 'ssh sdumont18 "pkill -u ${USER} -f jupyter"'

In [12]:
ssh -O exit sd

Exit request sent.


## Mostra processos ativos nos nós

In [17]:
ssh sd 'ssh sdumont11 "ps -u ${USER}"; exit'

  PID TTY          TIME CMD
10023 ?        00:00:32 jupyter-lab
10377 ?        00:00:04 python
10380 ?        00:00:03 python
10381 ?        00:00:03 python
10382 ?        00:00:04 python
29037 ?        00:00:00 sshd
29038 ?        00:00:00 ps


In [18]:
ssh sd 'ssh sdumont13 "ps -u ${USER}"; exit'

  PID TTY          TIME CMD
19884 ?        00:00:00 sshd
19885 ?        00:00:00 ps


In [19]:
ssh sd 'ssh sdumont14 "ps -u ${USER}"; exit'

  PID TTY          TIME CMD
 2857 ?        00:00:00 sftp-server
 2882 ?        00:00:00 sftp-server
19512 ?        00:00:00 sshd
24424 ?        00:00:00 python
24427 ?        00:00:00 python
24428 ?        00:00:00 python
28707 ?        00:00:05 jupyter-lab
28800 ?        00:00:00 bash
29112 ?        00:00:00 ssh
29425 ?        00:00:00 sshd
29543 ?        00:00:00 ps
30930 pts/114  00:00:00 bash


In [20]:
ssh sd 'ssh sdumont18 "ps -u ${USER}"; exit'

   PID TTY          TIME CMD
158875 ?        00:00:02 jupyter-lab
159322 ?        00:00:00 sshd
159339 ?        00:00:00 ps


# Referências

* [Running Jupyter Notebooks on Remote Servers](https://towardsdatascience.com/running-jupyter-notebooks-on-reamote-servers-603fbcc256b3?gi=cb964f4376e0)
* [Automatically set up jupyter notebook on remote server ](https://gist.github.com/cdgreenidge/45f2269a0b019d5ac45ab9530d5d75ca)
* [Create ssh tunnel to remote docker container running Jupyter Notebook](https://stackoverflow.com/questions/37576193/create-ssh-tunnel-to-remote-docker-container-running-jupyter-notebook)
